In [1]:
import uuid
import time
import tempfile
import ifcopenshell
import ifcopenshell.guid
import os
import numpy as np
import re
import glob
import pdb

O = 0.0, 0.0, 0.0
X = 1.0, 0.0, 0.0
Y = 0.0, 1.0, 0.0
Z = 0.0, 0.0, 1.0


# Helper function definitions
# Creates an IfcAxis2Placement3D from Location, Axis and RefDirection specified as Python tuples
def create_ifcaxis2placement(ifcfile, point=O, dir1=Z, dir2=X):
    # print("Creating IfcCartesianPoint with:", point)
    point = ifcfile.createIfcCartesianPoint(point)
    dir1 = ifcfile.createIfcDirection(dir1)
    dir2 = ifcfile.createIfcDirection(dir2)
    axis2placement = ifcfile.createIfcAxis2Placement3D(point, dir1, dir2)
    # pdb.set_trace()
    # print("fn calling")
    # print("point %s" % point)
    # print("dir1 %s" % dir1)
    # print("dir2 %s" % dir2)
    # print("axis2placement %s" % axis2placement)
    return axis2placement


# pdb.set_trace()
# Creates an IfcLocalPlacement from Location, Axis and RefDirection, specified as Python tuples, and relative placement
def create_ifclocalplacement(ifcfile, point=O, dir1=Z, dir2=X, relative_to=None):
    axis2placement = create_ifcaxis2placement(ifcfile, point, dir1, dir2)
    ifclocalplacement2 = ifcfile.createIfcLocalPlacement(relative_to, axis2placement)
    # pdb.set_trace()
    return ifclocalplacement2


# Creates an IfcPolyLine from a list of points, specified as Python tuples
def create_ifcpolyline(ifcfile, point_list):
    ifcpts = []
    for point in point_list:
        point = ifcfile.createIfcCartesianPoint(point)
        ifcpts.append(point)
    polyline = ifcfile.createIfcPolyLine(ifcpts)
    # print("ifcpts %s" %ifcpts)
    #
    # print("polyline %s" %polyline)
    return polyline


# Creates an IfcExtrudedAreaSolid from a list of points, specified as Python tuples
def create_ifcextrudedareasolid(
        ifcfile, point_list, ifcaxis2placement, extrude_dir, extrusion
):
    polyline = create_ifcpolyline(ifcfile, point_list)
    ifcclosedprofile = ifcfile.createIfcArbitraryClosedProfileDef(
        "AREA", None, polyline
    )
    ifcdir = ifcfile.createIfcDirection(extrude_dir)
    ifcextrudedareasolid = ifcfile.createIfcExtrudedAreaSolid(
        ifcclosedprofile, ifcaxis2placement, ifcdir, extrusion
    )
    return ifcextrudedareasolid


create_guid = lambda: ifcopenshell.guid.compress(uuid.uuid1().hex)
# print("create_guid %s" %create_guid)
# IFC template creation
filename = "ivy_hello_wall_window_test.ifc"
timestamp = time.time()
timestring = time.strftime("%Y-%m-%dT%H:%M:%S", time.gmtime(timestamp))
creator = "Ya Wen"
organization = "UOC"
application, application_version = "IfcOpenShell", "0.5"
project_globalid, project_name = create_guid(), "Hello Wall"
# A template IFC file to quickly populate entity instances for an IfcProject with its dependencies
template = (
        """ISO-10303-21;
    HEADER;
    FILE_DESCRIPTION(('ViewDefinition [CoordinationView]'),'2;1');
    FILE_NAME('%(filename)s','%(timestring)s',('%(creator)s'),('%(organization)s'),'%(application)s','%(application)s','');
    FILE_SCHEMA(('IFC2X3'));
    ENDSEC;
    DATA;
    #1=IFCPERSON($,$,'%(creator)s',$,$,$,$,$);
    #2=IFCORGANIZATION($,'%(organization)s',$,$,$);
    #3=IFCPERSONANDORGANIZATION(#1,#2,$);
    #4=IFCAPPLICATION(#2,'%(application_version)s','%(application)s','');
    #5=IFCOWNERHISTORY(#3,#4,$,.ADDED.,$,#3,#4,%(timestamp)s);
    #6=IFCDIRECTION((1.,0.,0.));
    #7=IFCDIRECTION((0.,0.,1.));
    #8=IFCCARTESIANPOINT((0.,0.,0.));
    #9=IFCAXIS2PLACEMENT3D(#8,#7,#6);
    #10=IFCDIRECTION((0.,1.,0.));
    #11=IFCGEOMETRICREPRESENTATIONCONTEXT($,'Model',3,1.E-05,#9,#10);
    #12=IFCDIMENSIONALEXPONENTS(0,0,0,0,0,0,0);
    #13=IFCSIUNIT(*,.LENGTHUNIT.,$,.METRE.);
    #14=IFCSIUNIT(*,.AREAUNIT.,$,.SQUARE_METRE.);
    #15=IFCSIUNIT(*,.VOLUMEUNIT.,$,.CUBIC_METRE.);
    #16=IFCSIUNIT(*,.PLANEANGLEUNIT.,$,.RADIAN.);
    #17=IFCMEASUREWITHUNIT(IFCPLANEANGLEMEASURE(0.017453292519943295),#16);
    #18=IFCCONVERSIONBASEDUNIT(#12,.PLANEANGLEUNIT.,'DEGREE',#17);
    #19=IFCUNITASSIGNMENT((#13,#14,#15,#18));
    #20=IFCPROJECT('%(project_globalid)s',#5,'%(project_name)s',$,$,$,$,(#11),#19);
    ENDSEC;
    END-ISO-10303-21;
    """
        % locals()
)


temp_handle, temp_filename = tempfile.mkstemp(suffix=".ifc")
try:
    # pdb.set_trace()
    with open(temp_filename, "wb") as f:
        f.write(template.encode("utf-8"))  
finally:
    os.close(temp_handle)  
# Obtain references to instances defined in template
ifcfile = ifcopenshell.open(temp_filename)
owner_history = ifcfile.by_type("IfcOwnerHistory")[0]
project = ifcfile.by_type("IfcProject")[0]
context = ifcfile.by_type("IfcGeometricRepresentationContext")[0]
# IFC hierarchy creation
site_placement = create_ifclocalplacement(ifcfile)
site = ifcfile.createIfcSite(
    create_guid(),
    owner_history,
    "Site",
    None,
    None,
    site_placement,
    None,
    None,
    "ELEMENT",
    None,
    None,
    None,
    None,
    None,
)
# print("site %s" % site)

building_placement = create_ifclocalplacement(ifcfile, relative_to=site_placement)
building = ifcfile.createIfcBuilding(
    create_guid(),
    owner_history,
    "Building",
    None,
    None,
    building_placement,
    None,
    None,
    "ELEMENT",
    None,
    None,
    None,
)
storey_placement = create_ifclocalplacement(ifcfile, relative_to=building_placement)
elevation = 0.0
building_storey = ifcfile.createIfcBuildingStorey(
    create_guid(),
    owner_history,
    "Storey",
    None,
    None,
    storey_placement,
    None,
    None,
    "ELEMENT",
    elevation,
)
# print ("building storey")
# print (building_storey)
"""
container_storey = ifcfile.createIfcRelAggregates(create_guid(), owner_history, "Building Container", None, building, [building_storey])
container_site = ifcfile.createIfcRelAggregates(create_guid(), owner_history, "Site Container", None, site, [building])
container_project = ifcfile.createIfcRelAggregates(create_guid(), owner_history, "Project Container", None, project, [site])
"""


# Wall creation: Define the wall shape as a polyline axis and an extruded area solid


def read_facade_points(facade_filename):
    points = []
    with open(facade_filename, "r") as file:
        for line in file:
            x, y, z = map(float, line.strip().split())
            points.append((x, y, z))
    return points


thickness = 0.5  # thickness of wall
thickness_coordinate = np.array([0.0, thickness, 0.0])
facade_filename = "facade01.txt"  
facade_points = np.array(read_facade_points(facade_filename))  # point reading 
# print("points", facade_points)
# print(type(facade_points[1]), type(thickness_coordinate))
# define the bottom_profile through the facade points
A = facade_points[1]
B = facade_points[2]
C = facade_points[2] - thickness_coordinate
D = facade_points[1] - thickness_coordinate
bottom_profile = [A.tolist(), B.tolist(), C.tolist(), D.tolist(), A.tolist()]
# define the height
height = float((facade_points[0] - facade_points[1])[2])
print("height", height, "bottom_profile", bottom_profile)

wall_placement = create_ifclocalplacement(
    ifcfile, A.tolist(), (0.0, 0.0, 1.0), (1.0, 0.0, 0.0), storey_placement
)
# wall_placement = create_ifclocalplacement(ifcfile, A.tolist(), (0.0, 0.0, 1.0), (1.0, 0.0, 0.0),
#                                                          wall_placement)
# polyline = create_ifcpolyline(ifcfile, [(0.0, -2.0, 0.0), (1.0, -2.0, 0.0)])
# axis_representation = ifcfile.createIfcShapeRepresentation(context, "Axis", "Curve2D", [polyline])
# extrusion_placement = create_ifcaxis2placement(ifcfile, (0.0, 0.0, 0.0),(0.0, 0.0, 1.0), (1.0, 0.0, 0.0))
# point_list_extrusion_area = [(0.0, -.2, 0.0), (5.0, -.2, 0.0), (5.0, .2, 0.0), (0.0, .2, 0.0), (0.0, -.2, 0.0)]
# solid = create_ifcextrudedareasolid(ifcfile, point_list_extrusion_area,
#         extrusion_placement, (0.0, 0.0, 1.0), 3.0)
# define polyline -> axis_representation -> product_shape
polyline = create_ifcpolyline(ifcfile, [(0.0, 0.0, 0.0), (1.0, 0.0, 0.0)])
axis_representation = ifcfile.createIfcShapeRepresentation(
    context, "Axis", "Curve2D", [polyline]
)
print("A", type(A.astype(float)[0]))
# define extrusion placement
extrusion_placement = create_ifcaxis2placement(
    ifcfile, (0.0, 0.0, 0.0), (0.0, 0.0, 1.0), (1.0, 0.0, 0.0)
)
solid = create_ifcextrudedareasolid(
    ifcfile, bottom_profile, extrusion_placement, (0.0, 0.0, 1.0), height
)
# print("solidddddddddd %s" % solid.Depth)
body_representation = ifcfile.createIfcShapeRepresentation(
    context, "Body", "SweptSolid", [solid]
)
product_shape = ifcfile.createIfcProductDefinitionShape(
    None, None, [axis_representation, body_representation]
)
wall = ifcfile.createIfcWallStandardCase(
    create_guid(),
    owner_history,
    "Wall",
    "An awesome wall",
    None,
    wall_placement,
    product_shape,
    None,
)
# new wall - second test wall
# # Wall creation: Define the wall shape as a polyline axis and an extruded area solid
# wall_placement2 = create_ifclocalplacement(ifcfile, relative_to=storey_placement)
# polyline2 = create_ifcpolyline(ifcfile, [(0.0, 0.0, 0.0), (5.0, 0.0, 0.0)])
# axis_representation2 = ifcfile.createIfcShapeRepresentation(context, "Axis", "Curve2D", [polyline2])
# extrusion_placement2 = create_ifcaxis2placement(ifcfile, (0.2, 0.2, 0.0),
#         (0.0, 0.0, 1.0), (0.0, 1.0, 0.0))
# point_list_extrusion_area2 = [(0.0, -.2, 0.0), (5.0, -.2, 0.0), (5.0,
#     .2, 0.0), (0.0, .2, 0.0), (0.0, -.2, 0.0)]
# solid2 = create_ifcextrudedareasolid(ifcfile, point_list_extrusion_area2,
#         extrusion_placement2, (0.0, 0.0, 1.0), 3.0)
# print("solidddddddddd %s" % solid.Depth)
# body_representation2 = ifcfile.createIfcShapeRepresentation(context, "Body", "SweptSolid", [solid2])
# product_shape2 = ifcfile.createIfcProductDefinitionShape(None, None, [axis_representation2, body_representation2])
# wall2 = ifcfile.createIfcWallStandardCase(create_guid(), owner_history, "Wall", "An awesome wall", None, wall_placement2, product_shape2, None)

# Define and associate the wall material
material = ifcfile.createIfcMaterial("wall material")
material_layer = ifcfile.createIfcMaterialLayer(material, 0.2, None)
material_layer_set = ifcfile.createIfcMaterialLayerSet([material_layer], None)
material_layer_set_usage = ifcfile.createIfcMaterialLayerSetUsage(
    material_layer_set, "AXIS2", "POSITIVE", -0.1
)
ifcfile.createIfcRelAssociatesMaterial(
    create_guid(),
    owner_history,
    RelatedObjects=[wall],
    RelatingMaterial=material_layer_set_usage,
)
# Create and assign property set
property_values = [
    ifcfile.createIfcPropertySingleValue(
        "Reference",
        "Reference",
        ifcfile.create_entity("IfcText", "Describe the Reference"),
        None,
    ),
    ifcfile.createIfcPropertySingleValue(
        "IsExternal", "IsExternal", ifcfile.create_entity("IfcBoolean", True), None
    ),
    ifcfile.createIfcPropertySingleValue(
        "ThermalTransmittance",
        "ThermalTransmittance",
        ifcfile.create_entity("IfcReal", 2.569),
        None,
    ),
    ifcfile.createIfcPropertySingleValue(
        "IntValue", "IntValue", ifcfile.create_entity("IfcInteger", 2), None
    ),
]
property_set = ifcfile.createIfcPropertySet(
    create_guid(), owner_history, "Pset_WallCommon", None, property_values
)
ifcfile.createIfcRelDefinesByProperties(
    create_guid(), owner_history, None, None, [wall], property_set
)
# Add quantity information
quantity_values = [
    ifcfile.createIfcQuantityLength("Length", "Length of the wall", None, 5.0),
    ifcfile.createIfcQuantityArea(
        "Area", "Area of the front face", None, 5.0 * solid.Depth
    ),
    ifcfile.createIfcQuantityVolume(
        "Volume",
        "Volume of the wall",
        None,
        5.0 * solid.Depth * material_layer.LayerThickness,
    ),
]
# print("solid depth %s" %solid.Depth)
# print("material_layer %s" %material_layer.LayerThickness)
element_quantity = ifcfile.createIfcElementQuantity(
    create_guid(), owner_history, "BaseQuantities", None, None, quantity_values
)
ifcfile.createIfcRelDefinesByProperties(
    create_guid(), owner_history, None, None, [wall], element_quantity
)
# Create and associate an opening for the window in the wall

# opening_points_np = np.array(opening_points)
ifcfile.createIfcRelContainedInSpatialStructure(
    create_guid(),
    owner_history,
    "Building Storey Container",
    None,
    [wall],
    building_storey,
)


def read_opening_points(filepath):
    opening_points = []
    with open(filepath, "r") as file:
        for line in file:
            x, y, z = map(float, line.strip().split())
            opening_points.append((x, y, z))
    return opening_points


def process_files_generator(directory):
    """生成器：逐个文件处理并返回点坐标及其命名。"""
    pattern = os.path.join(directory, "cluster_*_rec.txt")
    files = sorted(glob.glob(pattern)) 
    for filepath in files:
        base_name = os.path.basename(filepath) 
        file_id = base_name.split("_")[1]
        file_id_formatted = file_id.zfill(3)  
        opening_points = read_opening_points(filepath)
        opening_name = f"opening_{file_id_formatted}" 
        window_name = f"window_{file_id_formatted}"
        yield {opening_name: opening_points, window_name: opening_points}  


all_openings = []
for opening_data in process_files_generator("rectangle_txt"):
    all_openings.append(opening_data)  

print(np.array(list(all_openings[0].values())[0]), len(all_openings))




# create opening
def create_opening_and_window(ifcfile, opening_name, window_name, opening_points):
    window_thickness = 0.2  # thickness of window
    window_thickness_coordinate = np.array([0.0, window_thickness, 0.0])
    # Create an opening
    print("OA", opening_points[3])
    OA = opening_points[3]
    OB = opening_points[0]
    OC = opening_points[0] - thickness_coordinate
    OD = opening_points[3] - thickness_coordinate
    opening_bottom_profile = [
        OA.tolist(),
        OB.tolist(),
        OC.tolist(),
        OD.tolist(),
        OA.tolist(),
    ]
    opening_height = float((opening_points[1] - opening_points[0])[2])
    print(
        "opening height",
        opening_height,
        "opening bottom_profile",
        opening_bottom_profile,
    )


    placement_x = A[0]  # /2
    placement = [float(placement_x), float(OA[1]), float(OA[2])]

    # opening_placement = create_ifclocalplacement(
    #     ifcfile, (OA- center).tolist(), (0.0, 0.0, 1.0), (1.0, 0.0, 0.0), wall_placement
    # )
    opening_placement = create_ifclocalplacement(
        ifcfile, placement, (0.0, 0.0, 1.0), (1.0, 0.0, 0.0), storey_placement
    )

    opening_extrusion_placement = create_ifcaxis2placement(
        ifcfile, (0.0, 0.0, 0.0), (0.0, 0.0, 1.0), (1.0, 0.0, 0.0)
    )
    opening_solid = create_ifcextrudedareasolid(
        ifcfile,
        opening_bottom_profile,
        opening_extrusion_placement,
        (0.0, 0.0, 1.0),
        opening_height,
    )
    opening_representation = ifcfile.createIfcShapeRepresentation(
        context, "Body", "SweptSolid", [opening_solid]
    )
    opening_shape = ifcfile.createIfcProductDefinitionShape(
        None, None, [opening_representation]
    )

    opening_element = ifcfile.createIfcOpeningElement(
        create_guid(),
        owner_history,
        opening_name,
        "An awesome opening",
        None,
        opening_placement,
        opening_shape,
        None,
    )
    ifcfile.createIfcRelVoidsElement(
        create_guid(), owner_history, None, None, wall, opening_element
    )

    # Create a Window
    WA = OD + window_thickness_coordinate
    WB = OC + window_thickness_coordinate
    WC = OC
    WD = OD
    window_bottom_profile = [
        WA.tolist(),
        WB.tolist(),
        WC.tolist(),
        WD.tolist(),
        WA.tolist(),
    ]
    window_placement = create_ifclocalplacement(
        ifcfile, (0.0, 0.0, 0.0), (0.0, 0.0, 1.0), (1.0, 0.0, 0.0), opening_placement
    )
    window_extrusion_placement = create_ifcaxis2placement(
        ifcfile, (0.0, 0.0, 0.0), (0.0, 0.0, 1.0), (1.0, 0.0, 0.0)
    )
    window_solid = create_ifcextrudedareasolid(
        ifcfile,
        window_bottom_profile,
        window_extrusion_placement,
        (0.0, 0.0, 1.0),
        opening_height,
    )
    window_representation = ifcfile.createIfcShapeRepresentation(
        context, "Body", "SweptSolid", [window_solid]
    )
    window_shape = ifcfile.createIfcProductDefinitionShape(
        None, None, [window_representation]
    )
    window_element = ifcfile.createIfcWindow(
        create_guid(),
        owner_history,
        window_name,
        "An awesome window",
        None,
        window_placement,
        window_shape,
        None,
        None,
    )
    ifcfile.createIfcRelFillsElement(
        create_guid(), owner_history, None, None, opening_element, window_element
    )
    # Relate the window and wall to the building storey
    ifcfile.createIfcRelContainedInSpatialStructure(
        create_guid(),
        owner_history,
        "Building Storey Container",
        None,
        [window_element],
        building_storey,
    )
    return opening_element, window_element


for op in all_openings:
    items = list(op.items())
    if len(items) == 2:  
        (opening_name, opening_points), (window_name, window_points) = items
        opening_points_array = np.array(opening_points)
        window_points_array = np.array(window_points)
        oa, we = create_opening_and_window(ifcfile, opening_name, window_name, opening_points_array)
    else:
        print("Error: Each entry in all_openings must have exactly two items.")






# create_openings_and_window_from_files("rectangle_txt")


opening_placement = create_ifclocalplacement(ifcfile, (0.5, 0.0, 1.0), (0.0, 0.0, 1.0), (1.0, 0.0, 0.0), wall_placement)
opening_extrusion_placement = create_ifcaxis2placement(ifcfile, (0.0, 0.0, 0.0), (0.0, 0.0, 1.0), (1.0, 0.0, 0.0))
point_list_opening_extrusion_area = [(0.0, -0.2, 0.0), (3.0, -0.2, 0.0), (3.0, 0.2, 0.0), (0.0, 0.2, 0.0), (0.0, -0.2, 0.0)]
opening_solid = create_ifcextrudedareasolid(ifcfile, point_list_opening_extrusion_area, opening_extrusion_placement, (0.0, 0.0, 1.0), 1.3)
opening_representation = ifcfile.createIfcShapeRepresentation(context, "Body", "SweptSolid", [opening_solid])
opening_shape = ifcfile.createIfcProductDefinitionShape(None, None, [opening_representation])
opening_element = ifcfile.createIfcOpeningElement(create_guid(), owner_history, "Opening", "An awesome opening", None, opening_placement, opening_shape, None)
ifcfile.createIfcRelVoidsElement(create_guid(), owner_history, None, None, wall, opening_element)
# Create a simplified representation for the Window
window_placement = create_ifclocalplacement(ifcfile, (0.0, 0.0, 0.0), (0.0, 0.0, 1.0), (1.0, 0.0, 0.0), opening_placement)
window_extrusion_placement = create_ifcaxis2placement(ifcfile, (0.0, 0.0, 0.0), (0.0, 0.0, 1.0), (1.0, 0.0, 0.0))
point_list_window_extrusion_area = [(0.0, -0.06, 0.0), (3.0, -0.06, 0.0), (3.0, 0.06, 0.0), (0.0, 0.06, 0.0), (0.0, -0.06, 0.0)]
window_solid = create_ifcextrudedareasolid(ifcfile, point_list_window_extrusion_area, window_extrusion_placement, (0.0, 0.0, 1.0), 1.3)
window_representation = ifcfile.createIfcShapeRepresentation(context, "Body", "SweptSolid", [window_solid])
window_shape = ifcfile.createIfcProductDefinitionShape(None, None, [window_representation])
window = ifcfile.createIfcWindow(create_guid(), owner_history, "Window", "An awesome window", None, window_placement, window_shape, None, None)

# window_shape = ifcfile.createIfcProductDefinitionShape(None, None, [window_representation])
window = ifcfile.createIfcWallStandardCase(create_guid(), owner_history, "Wall", "An awesome wall", None, window_placement, window_shape, None)
# Relate the window to the opening element
ifcfile.createIfcRelFillsElement(create_guid(), owner_history, None, None, opening_element, window)
# Relate the window and wall to the building storey

# Write the contents of the file to disk
ifcfile.write(filename)

height 21.988 bottom_profile [[74.662, 0.0, 4.934], [135.421, 0.0, 5.594], [135.421, -0.5, 5.594], [74.662, -0.5, 4.934], [74.662, 0.0, 4.934]]
A <class 'numpy.float64'>
[[107.025   0.     19.155]
 [107.025   0.     20.535]
 [106.335   0.     20.535]
 [106.335   0.     19.155]] 272
OA [106.335   0.     19.155]
opening height 1.379999999999999 opening bottom_profile [[106.335, 0.0, 19.155], [107.025, 0.0, 19.155], [107.025, -0.5, 19.155], [106.335, -0.5, 19.155], [106.335, 0.0, 19.155]]
OA [133.395   0.      7.125]
opening height 1.2300000000000004 opening bottom_profile [[133.395, 0.0, 7.125], [133.965, 0.0, 7.125], [133.965, -0.5, 7.125], [133.395, -0.5, 7.125], [133.395, 0.0, 7.125]]
OA [97.395  0.    11.115]
opening height 1.2599999999999998 opening bottom_profile [[97.395, 0.0, 11.115], [97.965, 0.0, 11.115], [97.965, -0.5, 11.115], [97.395, -0.5, 11.115], [97.395, 0.0, 11.115]]
OA [115.935   0.     15.105]
opening height 0.0 opening bottom_profile [[115.935, 0.0, 15.105], [115.935